In [1]:
import time
import numpy as np
import pandas as pd
from os.path import join
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

def read_data():
    folder = 'data'
    df = pd.read_csv(join(folder, '10.25.14 us and uk 900 - Values Transformed - No Date No Post.csv'))
    # move class to first column
    df = df[['Dx_Chosen'] + [c for c in df if c not in ['Dx_Chosen']]]  
    df['Onset_Age'].fillna(df['Onset_Age'].mean(), inplace=True)
    df['Dx_Chosen'].fillna('None', inplace=True)  # fill class column Nan with 'None'
    # drop Initial_Dx column, drop the last 3 columns as they only contain 1 unique value
    df = df.drop(['Initial_Dx','Dx_Change','New_Dx','Part3_5_FS_ARDS',
                 'Part3_5_FS_lung','Part3_5_still_ARDS_24p'], 1)  
    x = df.iloc[:, 1:]
    y = df.iloc[:, 0]
    return x, y

def get_discrete_features(x):
    features = list(x.columns.values)
    discrete_features = []
    for f in features:
        if x[f].dtype == np.object:
            discrete_features.append(f)
    return discrete_features

start = time.time()
x, y = read_data()
dis_f = get_discrete_features(x)

x_dummies = pd.get_dummies(x, columns=dis_f, dummy_na=True)  # one hot encoding
#x_dummies.isnull().sum()

x_train, x_test, y_train, y_test = train_test_split(x_dummies, y, test_size=0.2, random_state=0)
rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
print(classification_report(y_test, y_pred))
print('--- running time: %.4f seconds ---' % (time.time() - start))

              precision    recall  f1-score   support

  ankylosing       0.87      0.94      0.90        35
   psoriatic       0.73      0.79      0.76        14
  rheumatoid       0.86      0.82      0.84        39
     sjogren       0.82      0.58      0.68        24
       still       1.00      0.88      0.94        17
    systemic       0.86      0.98      0.92        51

    accuracy                           0.86       180
   macro avg       0.86      0.83      0.84       180
weighted avg       0.86      0.86      0.86       180

--- running time: 0.7039 seconds ---


In [2]:
from sklearn.svm import SVC
start = time.time()
clf = SVC(gamma='auto')
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))
print('--- running time: %.4f seconds ---' % (time.time() - start))

              precision    recall  f1-score   support

  ankylosing       0.83      0.69      0.75        35
   psoriatic       0.00      0.00      0.00        14
  rheumatoid       0.36      0.62      0.45        39
     sjogren       0.00      0.00      0.00        24
       still       0.00      0.00      0.00        17
    systemic       0.48      0.78      0.59        51

    accuracy                           0.49       180
   macro avg       0.28      0.35      0.30       180
weighted avg       0.37      0.49      0.41       180

--- running time: 2.2786 seconds ---


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [3]:
from sklearn.neural_network import MLPClassifier
start = time.time()
mlp = MLPClassifier(hidden_layer_sizes=(60, 60, 60), random_state=0)
mlp.fit(x_train, y_train)
y_pred = mlp.predict(x_test)
print(classification_report(y_test, y_pred))
print('--- running time: %.4f seconds ---' % (time.time() - start))

              precision    recall  f1-score   support

  ankylosing       0.86      0.89      0.87        35
   psoriatic       0.72      0.93      0.81        14
  rheumatoid       0.84      0.79      0.82        39
     sjogren       0.77      0.71      0.74        24
       still       1.00      0.82      0.90        17
    systemic       0.87      0.90      0.88        51

    accuracy                           0.84       180
   macro avg       0.84      0.84      0.84       180
weighted avg       0.85      0.84      0.84       180

--- running time: 1.7613 seconds ---


### feature importance

In [16]:
features = x_dummies.columns
def get_rank(importances):
    indices = np.argsort(importances)[::-1]   
    for i in range(15):
        print("%2d) %-*s %f" % (i + 1, 30, features[indices[i]], importances[indices[i]]))

importances = rf.feature_importances_
get_rank(importances)

 1) Part3_3_sys_nan                0.048887
 2) Part3_6_Ank_nan                0.039398
 3) Part3_3_sys_yes                0.030963
 4) Part3_1_rheu_nan               0.023460
 5) Onset_Age                      0.012557
 6) Part3_4_sjo_nan                0.012134
 7) Part3_3_sys_other_Text_nan     0.012032
 8) Part3_6_Ank_yes,_i_was_told_for_the_first_time_prior_to_or_at_diagnosis_that_my_blood_tests_were_positive_for_the_hla-b27_gene. 0.011974
 9) Country_united_kingdom         0.010340
10) Part3_2_psor_nan               0.009538
11) Ex5_30c_Ps_Dx_yes              0.007610
12) Ex5_30d_Ps_pre_arthritis_yes   0.007445
13) Ex5_30c_Ps_Dx_no               0.007007
14) Ex5_30_Rash_butterfly_yes      0.006994
15) Country_united_states          0.006904


In [17]:
from sklearn.preprocessing import scale
#from sklearn.preprocessing import MinMaxScaler
"""
reference: https://stackoverflow.com/questions/35249760/
using-scikit-to-determine-contributions-of-each-feature-to-a-specific-class-pred/35255612
"""
def class_feature_importance(X, Y, feature_importances):
    N, M = X.shape
    X = scale(X)
    out = {}
    for c in set(Y):
        out[c] = np.mean(X[Y==c, :], axis=0)*feature_importances
    return out

result = class_feature_importance(x_dummies, y, importances)
result = pd.DataFrame(result)
result

,still,systemic,ankylosing,rheumatoid,psoriatic,sjogren
0,0.000467,-0.002434,-0.007365,0.005093,0.001869,0.004070
1,-0.000837,0.000330,-0.000734,0.000265,0.000007,0.000284
2,0.000833,-0.000328,0.000731,-0.000264,-0.000007,-0.000283
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-0.001718,0.008548,-0.004271,-0.002063,-0.004454,-0.003819
...,...,...,...,...,...,...
2422,0.000377,0.000003,-0.000042,-0.000147,0.000066,0.000032
2423,-0.000405,-0.000054,0.000044,0.000054,-0.000028,0.000200
2424,-0.000140,0.000061,-0.000005,-0.000033,-0.000037,0.000042
2425,-0.000244,-0.000090,0.000094,0.000286,0.000026,-0.000286


In [25]:
still = list(result['still'])
get_rank(still)

 1) Part3_3_sys_nan                0.029564
 2) Part3_6_Ank_nan                0.018041
 3) Ex5_19_Fv_Type_high,_spiking_fevers_averaging_>_102_degrees_f 0.009887
 4) Part3_15_still_No_Remember_yes 0.009081
 5) Part3_1_rheu_nan               0.008480
 6) Ex5_19_Fv_wsalmon_rash_yes     0.006323
 7) Ex5_30c_Ps_Dx_no               0.006128
 8) Ex5_30b_salmon_yes             0.005497
 9) Part3_3_sys_other_Text_nan     0.004899
10) Ex5_30_Rash_24_yes,_i_first_noticed_rashes_between_0_and_<_12_months_from_initial_onset 0.004254
11) Part3_4_sjo_nan                0.003551
12) Ex5_30_Rash_butterfly_yes      0.002152
13) Ex5_34_FS_Sore_Throat_yes      0.002079
14) Part3_2_psor_nan               0.002057
15) Ex5_19_Fv_Higher_0_12_yes      0.001773


In [26]:
systemic = list(result['systemic'])
get_rank(systemic)

 1) Part3_3_sys_yes                0.041125
 2) Part3_6_Ank_nan                0.018041
 3) Country_united_kingdom         0.008548
 4) Part3_1_rheu_nan               0.008480
 5) Ex5_30_Rash_butterfly_yes      0.004041
 6) Ex5_30c_Ps_Dx_no               0.003834
 7) Part3_4_sjo_nan                0.003551
 8) Part3_2_psor_nan               0.002057
 9) Part3_3_sys_no                 0.001697
10) Ex5_30_Rash_24_yes,_i_first_noticed_rashes_between_0_and_<_12_months_from_initial_onset 0.001525
11) Ex5_30d_Ps_pre_arthritis_nan   0.001448
12) Ex5_30b_sunburn_yes            0.001132
13) Ex5_34_CSores_Mouth_24_yes     0.000970
14) Ex4_7_Sac_0_12_no              0.000806
15) Raynauds_14_yes                0.000694


In [27]:
ankylosing = list(result['ankylosing'])
get_rank(ankylosing)

 1) Part3_3_sys_nan                0.029564
 2) Part3_6_Ank_yes,_i_was_told_for_the_first_time_prior_to_or_at_diagnosis_that_my_blood_tests_were_positive_for_the_hla-b27_gene. 0.017923
 3) Part3_1_rheu_nan               0.008480
 4) Part3_3_sys_other_Text_nan     0.004899
 5) Part3_4_sjo_nan                0.003551
 6) Ex5_30c_Ps_Dx_nan              0.002787
 7) Country_united_states          0.002347
 8) Ex2_3_RHand_ABCJoint_0_12_nan  0.002295
 9) Ex2_3_0_12_None_yes            0.002269
10) Ex4_7_FS_Sac_yes               0.002195
11) Part3_2_psor_nan               0.002057
12) Part3_6_Ank_yes,_i_was_told_for_the_first_time_after_diagnosis_that_my_blood_tests_were_positive_for_the_hla-b27_gene. 0.002031
13) Ex2_3_Rhand_0_12_no            0.002009
14) Ex5_30_Rash_butterfly_no       0.001890
15) Ex5_34_Thirst_24_no            0.001765


In [28]:
rheumatoid = list(result['rheumatoid'])
get_rank(rheumatoid)

 1) Part3_3_sys_nan                0.029564
 2) Part3_6_Ank_nan                0.018041
 3) Onset_Age                      0.005093
 4) Part3_3_sys_other_Text_nan     0.004731
 5) Part3_4_sjo_nan                0.003551
 6) Ex5_30c_Ps_Dx_nan              0.002209
 7) Part3_2_psor_nan               0.002057
 8) Ex2_3_Rhand_0_12_yes           0.001427
 9) Ex5_19_Fv_wsalmon_rash_no      0.001425
10) Ex5_30d_Ps_pre_arthritis_nan   0.001425
11) Ex5_30_Rash_butterfly_no       0.001230
12) Ex5_12_Sw_24_yes,_i_noticed_swelling_in_the_locations_i_experienced_paintenderness_between_0_and_<_12_months_from_initial_onset 0.001209
13) Part3_1_rheu_no                0.001142
14) Ex5_11_Red_Wm_24_yes,_i_noticed_rednesswarmth_between_0_and_<_12_months_from_initial_onset 0.001134
15) num_specialists_unrelated_0.  The first specialist I was referred to was a rheumatologist or immunologist 0.001119


In [31]:
psoriatic = list(result['psoriatic'])
get_rank(psoriatic)

 1) Part3_3_sys_nan                0.029564
 2) Part3_6_Ank_nan                0.018041
 3) Ex5_30c_Ps_Dx_yes              0.014172
 4) Ex5_30d_Ps_pre_arthritis_yes   0.011227
 5) Part3_1_rheu_nan               0.008480
 6) Ex5_30b_psoriasis_yes          0.003716
 7) Part3_4_sjo_nan                0.003551
 8) Part3_3_sys_other_Text_nan     0.003433
 9) Country_united_states          0.003077
10) Onset_Age                      0.001869
11) Ex5_19_Fv_wsalmon_rash_no      0.001864
12) Ex5_30b_salmon_no              0.001836
13) Ex5_30_Rash_butterfly_no       0.001834
14) Ex5_38_Pitted_nails_yes        0.001539
15) Ex5_30_Rash_Elb_yes            0.001535


In [32]:
sjogren = list(result['sjogren'])
get_rank(sjogren)

 1) Part3_3_sys_nan                0.029564
 2) Part3_6_Ank_nan                0.018041
 3) Part3_1_rheu_nan               0.008480
 4) Part3_3_sys_other_Text_nan     0.004630
 5) Onset_Age                      0.004070
 6) Country_united_states          0.002826
 7) Ex5_34_Thirst_24_yes           0.002824
 8) Part3_2_psor_nan               0.002057
 9) Ex5_19_Fv_wsalmon_rash_no      0.001514
10) Ex5_33_FS_Dry_Eye_yes          0.001369
11) Ex5_34_FS_Thirst_yes           0.001267
12) Ex5_33_FS_Burn_Itch_yes        0.001192
13) Ex5_30d_Ps_pre_arthritis_nan   0.001088
14) Ex5_33_Eye_Dry_24_yes          0.001049
15) Ex5_11_Red_Wm_24_no            0.000887
